# 5. Hypothesis CFP Accounting
<body> <p  style = "font-family:georgia,garamond,serif;font-size:16px;text-align:justify"></body>
We estimate equation 2 and 3 and we obtain Table 4 and Table 5 . 

In [1]:
import io
import ast
import pandas as pd
import numpy as np

from sqlalchemy import create_engine 
from linearmodels.iv import AbsorbingLS, Interaction
import statsmodels.formula.api as smf

In [2]:
credentials = ast.literal_eval(open(r"C:\Users\pvilas\OneDrive - unizar.es\Python Investigación\credentials.txt", "r").read())
engine = create_engine(credentials['sqlalchemy'][0]+ 'ESGRefinitiv')

var_names =        [ ['return_on_assets', 'ROA'],
                     ['return_on_equity', 'ROE'],
                     ['log_net_sales_usd', 'Log Sales'],
                     ['log_total_assets_usd','Log Assets'],
                     ['total_liabilities_to_equity', 'Liabilities to Equity'],
                     ['total_liabilities_to_assets', 'Liabilities to Assets'],
                     ['additions_fixed_assets_to_assets', 'Additions Fixed Assets to Assets'],
                     ['esg_score_average', 'ESG Conventional'], 
                     ['environment_pillar_average', 'Environ. Conv.'],
                     ['social_pillar_average', 'Social Conv.'],
                     ['governance_pillar_average', 'Governance Conv.'],
                     ['overall_inclusive', 'ESG Inclusive'],
                     ['environmental_inclusive', 'Environ. Incl.'],
                     ['social_inclusive', 'Social Inclusive'],
                     ['governance_inclusive', 'Governance Incl.'] ,
                   ]  # Variable name in database and name in the paper


df = pd.read_sql('b31_accounting_isp_ssp_winsorized', engine)
df['constant']=1

In [3]:
models = {'model_1' : ['log_net_sales_usd', 'total_liabilities_to_assets', 'additions_fixed_assets_to_assets', 'constant'],
          'model_2' :['log_total_assets_usd', 'total_liabilities_to_equity', 'additions_fixed_assets_to_assets', 'constant']
         }

dependent_variables=[['return_on_assets'], ['return_on_equity']]
blocking_variables = [[],
                      ["year_accounting"], 
                      ["year_accounting", "industry"],
                      ["year_accounting",  "industry", "ds_country"],
                      ["year_accounting", "internal_id"]]
                      
all_scores = {'csp' : [['esg_score_average'], ['environment_pillar_average'], 
                       ['social_pillar_average'], ['governance_pillar_average']],
'inclusive' : [["overall_inclusive"], ["environmental_inclusive"], 
         ["social_inclusive"], ["governance_inclusive"]] }



## <strong>Regressions to test H1,H2, H3</strong>

In [4]:
# dataframes where stored the results of each regression
df_blocking = pd.DataFrame(index = ['year_accounting', 'industry', 'ds_country', 'internal_id'] )
df_exog = pd.DataFrame(index = ['score', "score_coef", 
                                         'size', 'size_coef',
                                         'capital_structure', 'capital_structure_coef',
                                         'capital_intensity', 'capital_intensity_coef',
                                         'intercept', 'intercept_coef'] )
df_model_fit = pd.DataFrame(index = ['observatios', 'rsquared', 'rsquared_adj', 'rsquared_no_effects'])
df_model_info = pd.DataFrame(index = ['score_type', 'dependent_variable', 'model'])

In [5]:
def fixed_effects_model (data, endog, exog, effects):
    
    
    
    all_variables = endog + exog + effects
    df = data[all_variables].copy()
    df.dropna(inplace=True)
    # fixed effects to dataframe category
    for effect in effects:
        df[effect]= df[effect].astype('category')
        
    if not effects:
        model = AbsorbingLS(df[endog], df[exog])
        results = model.fit(cov_type='robust')   
    else:
        model = AbsorbingLS(df[endog], df[exog], absorb=df[effects])
        results = model.fit(cov_type='robust')
        
    return results

def significance(p_val):
    if p_val <=0.05 and p_val > 0.01 :
        significance = '*'
    elif p_val <=0.01 and p_val > 0.001:
        significance = '**'
    elif p_val <=0.001 :
        significance = '**'
    else:
        significance= ''
    significance = significance + "\n" 
    return significance

In [6]:
n_model = 0 # models counter
for model in models:
    for score_type in all_scores:
        for score in all_scores[score_type]:
            for dependent in dependent_variables:    
                for block in blocking_variables:
                    
                    endog = dependent 
                    exog =  score + models[model] 
                    effects= block
                    res = fixed_effects_model (df, endog, exog, effects)
                    
                    # coeficients results
                    df_results = pd.concat([res.params, res.std_errors, res.pvalues, res.tstats], 
                                           keys = ["Parameter", "Std.Err.", 'P-value', 'T-stat'],  axis= 1)
                    # Results research format
                    df_results["research"] =  ( df_results["Parameter"].round(3).map('{:.3f}'.format).astype(str) + 
                                     df_results['P-value'].apply(significance) + '{:.2%}'.format(res.rsquared)
#                                      df_results["Std.Err."].round(3).map('{:.3f}'.format).astype(str).apply(lambda x:  "("+ x + ")" ) 
                                    )
                    params = df_results.reset_index()['index'].tolist()
                    research = df_results.reset_index()['research'].tolist()

                    coef_results = []
                    for i in range (0, len(params)):
                        coef_results.append(params[i])
                        coef_results.append(research[i])

                    df_exog[n_model] = coef_results

                    # Model estimation goodness
                    df_model_fit[n_model] =[np.round(res.nobs,0), np.round(res.rsquared,2),
                                            np.round(res.rsquared_adj,2), np.round(res._absorbed_rsquared,2)]
                    #Model info
                    df_model_info[n_model] = [score_type, dependent[0], model]

                    # Blocking factors used (Name of the different blocking factors used)
                    block_used = list(dict.fromkeys(
                        var_block for group_block in blocking_variables for var_block in group_block))
                    for i in block_used:
                        if i in block:
                            block_used[block_used.index(i)]='Yes'
                        else:
                            block_used[block_used.index(i)]='No'
                    df_blocking[n_model] = block_used    

                    n_model += 1
                    
df_all_regresions = pd.concat([df_model_info.T, df_blocking.T, df_exog.T, df_model_fit.T ], axis=1, join='inner' )

<ipython-input-6-11218227692b>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_exog[n_model] = coef_results
<ipython-input-6-11218227692b>:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_model_fit[n_model] =[np.round(res.nobs,0), np.round(res.rsquared,2),
<ipython-input-6-11218227692b>:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-f

## <strong>Table 4, Table 5</strong>

In [7]:
df_table_all =df_all_regresions.pivot_table(values= "score_coef", index=['score_type', 'score', 'dependent_variable', 'model',], 
                                 columns =['year_accounting', 'industry', 'ds_country', 'internal_id'],
                                aggfunc='first')
for score_type in all_scores:
    df_table = df_table_all.loc[(score_type),].copy()
    
    score_index = list(df_table.index.get_level_values(0))
    score_order = [t[0]  for t in var_names if t[0] in score_index]    
    
    df_table= df_table.sort_index(axis=1,level=[3,2,1,0])
    
    df_table=df_table.reindex(score_order, level=0)
    
    

#     for var in var_names:
#         df_table = df_table.rename(columns={var[0]:var[1]}, index={var[0]:var[1]})
    df_table.to_excel(r'C:\Users\pvilas\OneDrive - unizar.es\Tesis\2022 ISP\3. Tables & Figures\3. Definitive\Table %s.xlsx' %(str(4+list(all_scores).index(score_type))+" "+ score_type) )

df_table

year_accounting                                                  No  \
industry                                                         No   
ds_country                                                       No   
internal_id                                                      No   
score                   dependent_variable model                      
overall_inclusive       return_on_assets   model_1  -0.030**\n7.57%   
                                           model_2  -0.029**\n4.35%   
                        return_on_equity   model_1  -0.074**\n3.63%   
                                           model_2  -0.056**\n0.99%   
environmental_inclusive return_on_assets   model_1  -0.027**\n7.75%   
                                           model_2  -0.024**\n4.45%   
                        return_on_equity   model_1  -0.066**\n3.81%   
                                           model_2  -0.054**\n1.14%   
social_inclusive        return_on_assets   model_1  -0.028**\n7.63%   
                                           model_2  -0.027**\n4.40%   
                        return_on_equity   model_1  -0.068**\n3.68%   
                                           model_2  -0.056**\n1.06%   
governance_inclusive    return_on_assets   model_1  -0.006**\n7.22%   
                                           model_2  -0.006**\n4.01%   
                        return_on_equity   model_1  -0.017**\n3.29%   
                                           model_2   -0.009*\n0.78%   

year_accounting                                                 Yes  \
industry                                                         No   
ds_country                                                       No   
internal_id                                                      No   
score                   dependent_variable model                      
overall_inclusive       return_on_assets   model_1  -0.029**\n8.09%   
                                           model_2  -0.029**\n5.04%   
                        return_on_equity   model_1  -0.073**\n4.13%   
                                           model_2  -0.055**\n1.62%   
environmental_inclusive return_on_assets   model_1  -0.027**\n8.27%   
                                           model_2  -0.024**\n5.14%   
                        return_on_equity   model_1  -0.065**\n4.30%   
                                           model_2  -0.053**\n1.76%   
social_inclusive        return_on_assets   model_1  -0.028**\n8.16%   
                                           model_2  -0.027**\n5.10%   
                        return_on_equity   model_1  -0.068**\n4.17%   
                                           model_2  -0.056**\n1.69%   
governance_inclusive    return_on_assets   model_1  -0.006**\n7.75%   
                                           model_2  -0.006**\n4.72%   
                        return_on_equity   model_1  -0.017**\n3.79%   
                                           model_2   -0.008*\n1.41%   

year_accounting                                                       \
industry                                                         Yes   
ds_country                                                        No   
internal_id                                                       No   
score                   dependent_variable model                       
overall_inclusive       return_on_assets   model_1  -0.026**\n12.35%   
                                           model_2   -0.027**\n9.39%   
                        return_on_equity   model_1   -0.062**\n8.13%   
                                           model_2   -0.050**\n6.31%   
environmental_inclusive return_on_assets   model_1  -0.024**\n12.53%   
                                           model_2   -0.023**\n9.48%   
                        return_on_equity   model_1   -0.058**\n8.29%   
                                           model_2   -0.049**\n6.43%   
social_inclusive        return_on_assets   model_1  -0.026**\n12.44%   
        